In [15]:
import pandas as pd
import numpy as np

# --- 1. DEFINE FILE NAMES ---
file_main = "Banking.csv"
file_clients = "banking-clients.csv"
file_gender = "gender.csv"
file_relationship = "banking-realtionships.csv" # Using your exact file name
file_advisor = "investment-advisiors.csv"     # Using your exact file name

try:
    # --- 2. LOAD DATA ---
    df_main = pd.read_csv(file_main)
    df_clients = pd.read_csv(file_clients)
    df_gender = pd.read_csv(file_gender)
    df_relationship = pd.read_csv(file_relationship)
    df_advisor = pd.read_csv(file_advisor)
    
    print("--- Step 1: All 5 files loaded successfully. ---")

    # --- 3. MERGE DATAFRAMES ---
    df_merged = df_main.merge(df_clients, on='Client ID', how='left')
    df_merged = df_merged.merge(df_gender, on='Gender ID', how='left')
    df_merged = df_merged.merge(df_relationship, on='Banking Relationship ID', how='left')
    df_merged = df_merged.merge(df_advisor, on='Investment Advisor ID', how='left')
    
    print("--- Step 2: All tables merged successfully. ---")

    # --- 4. CLEAN & TRANSFORM (FEATURE ENGINEERING) ---
    
    # Clean financial columns (fill NaNs with 0 before math)
    financial_cols = [
        'Bank Loans', 'Business Lending', 'Credit Card Balance', 
        'Bank Deposits', 'Saving Accounts', 'Foreign Currency Account', 'Checking Accounts'
    ]
    for col in financial_cols:
        df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce').fillna(0)

    # A. Create 'Engagment Days'
    df_merged['Joined Bank'] = pd.to_datetime(df_merged['Joined Bank'], errors='coerce')
    df_merged['Engagment Days'] = (pd.Timestamp.today() - df_merged['Joined Bank']).dt.days

    # B. Create 'Engagement Timeframe'
    bins_time = [-float('inf'), 365, 1825, 3650, 7300, float('inf')]
    labels_time = ["< 1 Years", "< 5 Years", "< 10 Years", "< 20 Years", "> 20 Years"]
    df_merged['Engagement Timeframe'] = pd.cut(df_merged['Engagment Days'], bins=bins_time, labels=labels_time, right=False)

    # C. Create 'Income Band'
    df_merged['Estimated Income'] = pd.to_numeric(df_merged['Estimated Income'], errors='coerce').fillna(0)
    bins_income = [-float('inf'), 100000, 300000, float('inf')]
    labels_income = ["Low", "Mid", "High"]
    df_merged['Income Band'] = pd.cut(df_merged['Estimated Income'], bins=bins_income, labels=labels_income, right=False)

    # D. Create 'Processing Fees'
    fee_map = { "High": 0.05, "Mid": 0.03, "Low": 0.01 }
    df_merged['Processing Fees'] = df_merged['Fee Structure'].map(fee_map).fillna(0)

    # E. Create 'Total Loan'
    df_merged['Total Loan'] = df_merged['Bank Loans'] + df_merged['Business Lending'] + df_merged['Credit Card Balance']

    # F. Create 'Total Deposit'
    df_merged['Total Deposit'] = df_merged['Bank Deposits'] + df_merged['Saving Accounts'] + \
                                 df_merged['Foreign Currency Account'] + df_merged['Checking Accounts']
                                 
    print("--- Step 3: All new columns created. ---")

    # --- 5. FINALIZE ---
    # Drop the old ID columns now that we have the text names
    cols_to_drop = ['Gender ID', 'Banking Relationship ID', 'Investment Advisor ID']
    df_final = df_merged.drop(columns=cols_to_drop)
    
    print("--- Step 4: Redundant ID columns dropped. ---")
    
    # --- 6. DISPLAY RESULTS ---
    print("\n\n--- FINAL DATAFRAME INFO ---")
    df_final.info()
    
    print("\n\n--- FINAL DATAFRAME HEAD (First 5 Rows) ---")
    print(df_final.head())

except FileNotFoundError as e:
    print(f"Error: {e}. One of the 5 data files was not found.")
except KeyError as e:
    print(f"KeyError: {e}. A column name for merging is incorrect.")
except Exception as e:
    print(f"An error occurred: {e}")

--- Step 1: All 5 files loaded successfully. ---
KeyError: 'Gender ID'. A column name for merging is incorrect.


In [ ]:
import pandas as pd
import numpy as np

# --- 1. DEFINE FILE NAMES ---
file_main = "Banking.csv"
file_clients = "banking-clients.csv"
file_gender = "gender.csv"
file_relationship = "banking-realtionships.csv"
file_advisor = "investment-advisiors.csv"

try:
    # --- 2. LOAD DATA ---
    df_main = pd.read_csv(file_main)
    df_clients = pd.read_csv(file_clients)
    df_gender = pd.read_csv(file_gender)
    df_relationship = pd.read_csv(file_relationship)
    df_advisor = pd.read_csv(file_advisor)
    
    print("--- Step 1: All 5 files loaded successfully. ---")

    # --- 3. MERGE DATAFRAMES (with correction) ---
    df_merged = df_main.merge(df_clients, on='Client ID', how='left')
    
    # --- THIS IS THE FIX ---
    # Merge Gender using left_on and right_on because the names are different
    df_merged = df_merged.merge(
        df_gender, 
        left_on='Gender ID',    # Column from df_main (Banking.csv)
        right_on='Gender ID',  # Column from df_gender (gender.csv)
        how='left'
    )
    
    df_merged = df_merged.merge(df_relationship, on='Banking Relationship ID', how='left')
    df_merged = df_merged.merge(df_advisor, on='Investment Advisor ID', how='left')
    
    print("--- Step 2: All tables merged successfully. ---")

    # --- 4. CLEAN & TRANSFORM (FEATURE ENGINEERING) ---
    
    # Clean financial columns (fill NaNs with 0)
    financial_cols = [
        'Bank Loans', 'Business Lending', 'Credit Card Balance', 
        'Bank Deposits', 'Saving Accounts', 'Foreign Currency Account', 'Checking Accounts'
    ]
    for col in financial_cols:
        df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce').fillna(0)

    # A. Create 'Engagment Days'
    df_merged['Joined Bank'] = pd.to_datetime(df_merged['Joined Bank'], errors='coerce')
    df_merged['Engagment Days'] = (pd.Timestamp.today() - df_merged['Joined Bank']).dt.days

    # B. Create 'Engagement Timeframe'
    bins_time = [-float('inf'), 365, 1825, 3650, 7300, float('inf')]
    labels_time = ["< 1 Years", "< 5 Years", "< 10 Years", "< 20 Years", "> 20 Years"]
    df_merged['Engagement Timeframe'] = pd.cut(df_merged['Engagment Days'], bins=bins_time, labels=labels_time, right=False)

    # C. Create 'Income Band'
    df_merged['Estimated Income'] = pd.to_numeric(df_merged['Estimated Income'], errors='coerce').fillna(0)
    bins_income = [-float('inf'), 100000, 300000, float('inf')]
    labels_income = ["Low", "Mid", "High"]
    df_merged['Income Band'] = pd.cut(df_merged['Estimated Income'], bins=bins_income, labels=labels_income, right=False)

    # D. Create 'Processing Fees'
    fee_map = { "High": 0.05, "Mid": 0.03, "Low": 0.01 }
    df_merged['Processing Fees'] = df_merged['Fee Structure'].map(fee_map).fillna(0)

    # E. Create 'Total Loan'
    df_merged['Total Loan'] = df_merged['Bank Loans'] + df_merged['Business Lending'] + df_merged['Credit Card Balance']

    # F. Create 'Total Deposit'
    df_merged['Total Deposit'] = df_merged['Bank Deposits'] + df_merged['Saving Accounts'] + \
                                 df_merged['Foreign Currency Account'] + df_merged['Checking Accounts']
                                 
    print("--- Step 3: All new columns created. ---")

    # --- 5. FINALIZE ---
    # Drop the old ID columns
    cols_to_drop = [
        'Gender ID', # Original from Banking.csv
        'GenderID',  # Original from gender.csv
        'Banking Relationship ID', 
        'Investment Advisor ID'
    ]
    df_final = df_merged.drop(columns=cols_to_drop)
    
    print("--- Step 4: Redundant ID columns dropped. ---")
    
    # --- 6. DISPLAY RESULTS ---
    print("\n\n--- FINAL DATAFRAME INFO ---")
    df_final.info()
    
    print("\n\n--- FINAL DATAFRAME HEAD (First 5 Rows) ---")
    print(df_final.head())

except FileNotFoundError as e:
    print(f"Error: {e}. One of the 5 data files was not found.")
except KeyError as e:
    print(f"KeyError: {e}. A column name for merging is incorrect.")
except Exception as e:
    print(f"An error occurred: {e}")

--- Step 1: All 5 files loaded successfully. ---
KeyError: 'GenderID'. A column name for merging is incorrect.


In [17]:
import pandas as pd

print("--- Inspecting columns for all 5 key files ---")

# List of the 5 files we identified as critical
key_files = [
    "Banking.csv",
    "banking-clients.csv",
    "gender.csv",
    "banking-realtionships.csv",
    "investment-advisiors.csv"
]

for file in key_files:
    try:
        # Read only the first row to get headers
        df = pd.read_csv(file, nrows=0) 
        print(f"\n--- File: {file} ---")
        print("Columns:")
        # Print columns as a list for clarity
        print(df.columns.tolist())
    except FileNotFoundError:
        print(f"\n--- File: {file} ---")
        print("!!! FILE NOT FOUND !!!")
    except Exception as e:
        print(f"\n--- File: {file} ---")
        print(f"Error reading file: {e}")

--- Inspecting columns for all 5 key files ---

--- File: Banking.csv ---
Columns:
['Client ID', 'Name', 'Age', 'Location ID', 'Joined Bank', 'Banking Contact', 'Nationality', 'Occupation', 'Fee Structure', 'Loyalty Classification', 'Estimated Income', 'Superannuation Savings', 'Amount of Credit Cards', 'Credit Card Balance', 'Bank Loans', 'Bank Deposits', 'Checking Accounts', 'Saving Accounts', 'Foreign Currency Account', 'Business Lending', 'Properties Owned', 'Risk Weighting', 'BRId', 'GenderId', 'IAId']

--- File: banking-clients.csv ---
Columns:
['Client ID', 'Name', 'Age', 'Location ID', 'Joined Bank', 'Banking Contact', 'Nationality', 'Occupation', 'Fee Structure', 'Loyalty Classification', 'Estimated Income', 'Superannuation Savings', 'Amount of Credit Cards', 'Credit Card Balance', 'Bank Loans', 'Bank Deposits', 'Checking Accounts', 'Saving Accounts', 'Foreign Currency Account', 'Business Lending', 'Properties Owned', 'Risk Weighting', 'BRId', 'GenderId', 'IAId']

--- File: ge

In [18]:
import pandas as pd
import numpy as np

# --- 1. DEFINE FILE NAMES ---
# We only need 4 files. Banking.csv is our main table.
file_main = "Banking.csv"
file_gender = "gender.csv"
file_relationship = "banking-realtionships.csv"
file_advisor = "investment-advisiors.csv"

try:
    # --- 2. LOAD DATA ---
    df_main = pd.read_csv(file_main)
    df_gender = pd.read_csv(file_gender)
    df_relationship = pd.read_csv(file_relationship)
    df_advisor = pd.read_csv(file_advisor)
    
    print("--- Step 1: All 4 necessary files loaded successfully. ---")
    print("(Note: 'banking-clients.csv' is a duplicate of 'Banking.csv' and is not needed.)")


    # --- 3. MERGE DATAFRAMES (with correct keys) ---
    # We will merge the 3 small tables into our main 'df_main' table.
    
    df_merged = df_main.merge(df_gender, on='GenderId', how='left')
    df_merged = df_merged.merge(df_relationship, on='BRId', how='left')
    df_merged = df_merged.merge(df_advisor, on='IAId', how='left')
    
    print("--- Step 2: All tables merged successfully using 'GenderId', 'BRId', and 'IAId'. ---")

    # --- 4. CLEAN & TRANSFORM (FEATURE ENGINEERING) ---
    
    # Clean financial columns (fill NaNs with 0)
    financial_cols = [
        'Bank Loans', 'Business Lending', 'Credit Card Balance', 
        'Bank Deposits', 'Saving Accounts', 'Foreign Currency Account', 'Checking Accounts'
    ]
    for col in financial_cols:
        df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce').fillna(0)

    # A. Create 'Engagment Days'
    df_merged['Joined Bank'] = pd.to_datetime(df_merged['Joined Bank'], errors='coerce')
    df_merged['Engagment Days'] = (pd.Timestamp.today() - df_merged['Joined Bank']).dt.days

    # B. Create 'Engagement Timeframe'
    bins_time = [-float('inf'), 365, 1825, 3650, 7300, float('inf')]
    labels_time = ["< 1 Years", "< 5 Years", "< 10 Years", "< 20 Years", "> 20 Years"]
    df_merged['Engagement Timeframe'] = pd.cut(df_merged['Engagment Days'], bins=bins_time, labels=labels_time, right=False)

    # C. Create 'Income Band'
    df_merged['Estimated Income'] = pd.to_numeric(df_merged['Estimated Income'], errors='coerce').fillna(0)
    bins_income = [-float('inf'), 100000, 300000, float('inf')]
    labels_income = ["Low", "Mid", "High"]
    df_merged['Income Band'] = pd.cut(df_merged['Estimated Income'], bins=bins_income, labels=labels_income, right=False)

    # D. Create 'Processing Fees'
    fee_map = { "High": 0.05, "Mid": 0.03, "Low": 0.01 }
    df_merged['Processing Fees'] = df_merged['Fee Structure'].map(fee_map).fillna(0)

    # E. Create 'Total Loan'
    df_merged['Total Loan'] = df_merged['Bank Loans'] + df_merged['Business Lending'] + df_merged['Credit Card Balance']

    # F. Create 'Total Deposit'
    df_merged['Total Deposit'] = df_merged['Bank Deposits'] + df_merged['Saving Accounts'] + \
                                 df_merged['Foreign Currency Account'] + df_merged['Checking Accounts']
                                 
    print("--- Step 3: All new columns created. ---")

    # --- 5. FINALIZE ---
    # Drop the old ID columns
    cols_to_drop = ['GenderId', 'BRId', 'IAId']
    df_final = df_merged.drop(columns=cols_to_drop)
    
    print("--- Step 4: Redundant ID columns dropped. ---")
    
    # --- 6. DISPLAY RESULTS ---
    print("\n\n--- FINAL DATAFRAME INFO ---")
    df_final.info()
    
    print("\n\n--- FINAL DATAFRAME HEAD (First 5 Rows) ---")
    print(df_final.head())

except FileNotFoundError as e:
    print(f"Error: {e}. One of the 4 data files was not found.")
except KeyError as e:
    print(f"KeyError: {e}. A column name for merging is incorrect.")
except Exception as e:
    print(f"An error occurred: {e}")

--- Step 1: All 4 necessary files loaded successfully. ---
(Note: 'banking-clients.csv' is a duplicate of 'Banking.csv' and is not needed.)
--- Step 2: All tables merged successfully using 'GenderId', 'BRId', and 'IAId'. ---
--- Step 3: All new columns created. ---
--- Step 4: Redundant ID columns dropped. ---


--- FINAL DATAFRAME INFO ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Client ID                 3000 non-null   object        
 1   Name                      3000 non-null   object        
 2   Age                       3000 non-null   int64         
 3   Location ID               3000 non-null   int64         
 4   Joined Bank               1190 non-null   datetime64[ns]
 5   Banking Contact           3000 non-null   object        
 6   Nationality               3000 non-null   object 